In [76]:
import csv
import pandas as pd
import numpy as np

In [77]:
escitalopram_id = "CID100002771" #"CID100002771"
atc1 = "N06AB04"
atc2 = "N06AB10"

In [103]:
dataset_path = "/home/samuel/NYU/BDS/project/meddra/"

def read_tsv_file(file_path, headers = True):
    try:
        data = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            # Using the CSV reader with the tab delimiter
            reader = csv.reader(file, delimiter='\t')

            if headers == True:
                # Reading headers
                headers = next(reader)
                print("Headers:", headers)

            # Reading data rows
            for row in reader:
                data.append(row)

        # Create a Pandas DataFrame using the headers and data
        if headers == True:
            df = pd.DataFrame(data, columns=headers)
        else:
            df = pd.DataFrame(data) #, columns=headers)
        return df

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")
meddra_file_path = dataset_path + 'meddra_all_se.tsv'
meddra_df = read_tsv_file(meddra_file_path)
escitalopram_sides_table = meddra_df[meddra_df[0] == escitalopram_id]
escitalopram_sides = escitalopram_sides_table[5].unique()
processed_dataset = pd.read_csv("../review_dataset/drugsComTrain_processed.csv")
escitalopram_dataset = processed_dataset[processed_dataset["drugName"] == "Escitalopram"]

Headers: ['CID100000085', 'CID000010917', 'C0000729', 'LLT', 'C0000729', 'Abdominal cramps']


In [79]:
escitalopram_dataset

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review
203,140891,Escitalopram,Generalized Anxiety Disorde,"""The time I been on lexapro I had mild side ef...",7.0,"April 27, 2016",13,"""the time lexapro mild side effect like yawn i..."
237,214628,Escitalopram,Generalized Anxiety Disorde,"""This drug is pretty great for depression, how...",5.0,"August 14, 2017",7,"""thi drug pretti great depression, howev serio..."
252,141047,Escitalopram,Anxiety,"""Day 2. Taking for major anxiety/ slight depre...",10.0,"October 6, 2015",11,"""day 2. take major anxiety/ slight depression...."
314,142104,Escitalopram,Depression,"""I&#039;ve been on lexapro for several years. ...",8.0,"March 9, 2009",376,"""i'v lexapro sever years. say initi side effec..."
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per..."
...,...,...,...,...,...,...,...,...
160596,141107,Escitalopram,Depression,"""It works really good. I also have anxiety and...",9.0,"August 10, 2015",65,"""it work realli good. also anxieti part helps...."
160625,141426,Escitalopram,Anxiety,"""I feel compelled to share my experience becau...",9.0,"January 27, 2014",41,"""i feel compel share experi know other like me..."
160687,140846,Escitalopram,Depression,"""I have taken Lexapro for 10 years. I had seve...",9.0,"April 20, 2016",32,"""i taken lexapro 10 years. sever depression/an..."
160716,141419,Escitalopram,Depression,"""Sexually frustrated me and I was a little bit...",6.0,"February 9, 2014",36,"""sexual frustrat littl bit aggress come onto i..."


In [80]:
import pandas as pd

# Create a DataFrame with side effects and initialize the count to 0
escitalopram_sides_counted = pd.DataFrame({"side_effect_name": escitalopram_sides})

# Iterate through each review
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    review_lower = row["review"].lower()
    
    # Iterate through each side effect
    for side_effect in escitalopram_sides_counted["side_effect_name"]:
        # Replace side effect mentions with an empty string
        review_lower = review_lower.replace(side_effect.lower(), "")
        
    
    # Store the cleaned review in the dataframe
    escitalopram_dataset.loc[index, "cleaned_review"] = review_lower

# Print the first few rows of the dataframe to verify the results

/tmp/ipykernel_14141/2995517786.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[index, "cleaned_review"] = review_lower


In [81]:
escitalopram_dataset.iloc[0]["cleaned_review"]

'"the time i been on lexapro i had mild side effects like  and  really bad to the point i am only sleeping five hours or so...i can say it calms you but i haven&#039;t got the full effect of getting back to my old self again..."'

In [82]:
escitalopram_sides_counted = pd.DataFrame(escitalopram_sides, columns=["side_effect_name"]).copy()
escitalopram_sides_counted["count"] = 0
for review in escitalopram_dataset["review"]:
    for side_effect_name in escitalopram_sides_counted["side_effect_name"]:
        if side_effect_name.lower() in review.lower():
            escitalopram_sides_counted.loc[escitalopram_sides_counted["side_effect_name"] == side_effect_name, "count"] += 1
escitalopram_sides_counted.sort_values(by = "count", ascending=False)

,side_effect_name,count
17,Anxiety,785
355,Depression,439
607,Panic attack,213
183,Nausea,122
120,Headache,112
...,...,...
313,Phosphatase alkaline increased,0
314,Blood alkaline phosphatase increased,0
315,Prothrombin level increased,0
316,Electrocardiogram QT prolonged,0


In [6]:
relevant_side_effect = list(escitalopram_sides_counted[escitalopram_sides_counted["count"] > 5]["side_effect_name"])
for relevant_effect in relevant_side_effect:
    escitalopram_dataset.loc[:, relevant_effect] = False
    for index,  review in escitalopram_dataset["review"].items():
        if relevant_effect.lower() in review.lower():
            escitalopram_dataset.loc[index, relevant_effect] = True
reg_df = escitalopram_dataset.drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review"])


/tmp/ipykernel_14141/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_14141/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_14141/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [110]:
import re
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "years old" in review_orig:
        age_index = review_orig.find("years old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
            
    if "year-old" in review_orig:
        age_index = review_orig.find("year-old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()

    if "years-old" in review_orig:
        age_index = review_orig.find("years-old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
    if "year old" in review_orig:
        age_index = review_orig.find("year old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
    if "y/o" in review_orig:
        age_index = review_orig.find("y/o")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
            
    if "years of age" in review_orig:
        age_index = review_orig.find("years of age")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
found = escitalopram_dataset[escitalopram_dataset["age"] != "na"].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found ages: {found/total}")

ratio of found ages: 0.05959752321981424


In [113]:
import re
escitalopram_dataset["married"] = False
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "husband" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "wife" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "married" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "marriage" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "spouse" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
found = escitalopram_dataset[escitalopram_dataset["married"] != False].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found married: {found/total}")

ratio of found ages: 0.053405572755417956


/tmp/ipykernel_14141/2188157712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset["married"] = False


In [114]:
import re
escitalopram_dataset["children"] = False
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "child" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "children" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "kid" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "kids" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
found = escitalopram_dataset[escitalopram_dataset["children"] != False].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found children: {found/total}")

ratio of found children: 0.04411764705882353


/tmp/ipykernel_14141/1544857992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset["children"] = False
